# **Connectionist Temporal Classification**

O CTC (do inglês, *Connectionist Temporal Classification*) é algoritmo utilizado para treinar modelos profundos de redes neurais para tarefas como reconhecimento de fala e problemas de sequência. Em problemas de reconhecimento de fala um dos maiores problemas é entender quantas amostras do áudio que devem ser mapeadas para um certo caracter ou fonema, isto é, não existe alinhamento entre a sequência de entrada e a sequência de saída.

## **O problema**
Seja $X$, um áudio, isto é, $X = [x_1, x_2, ..., x_t]$ e $Y$ uma transcrição, isto é $Y = [y_1, y_2, ..., y_v]$. A nossa tarefa é encontrar $f$ tal que:

$$
f: X \rightarrow Y
$$

Tanto os elementos de $X$ quanto $Y$ podem variar de tamanho. Impossibilitando técnicas clássicas de aprendizado de máquina. O CTC contorna esse problema ao retornar uma distribuição de probabilidade de **Y** (de todo possível valor) para cada **X**.

Exemplo:

Uma entrada de tamanho 5 e $Y = [a,n,a]$ (dois possiveis tokens):

- Primeiro associamos cada $x$ ao elemento mais provável de $Y$.
- Colapsa os elementos repetidos.

| entrada:        |  x1 |  x2 |  x3 |  x4 |  x5 |
| :-              | :-: | :-: | :-: | :-: | :-: |
|**alinhamento**: | a   | n   | n   | a   | a   |
|**Saida**:       | a   | n   |     | a   |     |

Um problema com essa metodologia:

- Como podemos escrever palavras como Cachorro ou Osso ? Uma vez que quando colapsado teríamos Cachoro ou Oso.

Para resolver esse problema o algoritmo do CTC introduz um *token* $\epsilon$ normalmente denominado *blank token*. <font color ="red"> Esse Token não tem nenhuma correspondência (Ele é diferente do token espaço em branco)</font>.

Exemplo:

Uma entrada de tamanho 7 e $Y = [o,s,s,o]$ (três possiveis tokens - {o,s,$\epsilon$}):

| entrada:        | x1 |  x2 |  x3 |  x4 |  x5 | x6 | x7 |
| :-              | :-: | :-: | :-: | :-: | :-: |:-: | :-: |
|**alinhamento**: | o   | s   | $\epsilon$| s  | s | o  | o | o |
|**colapsa**:     | o   | s   | $\epsilon$| s  |   | o  |  |  |
|**remove** $\epsilon$:     | o   | s   | | s  |   | o  |  |  |


Existem vários alinhamentos possíveis, ainda no mesmo exemplo, algumas opções <font color="orange">**válidas**</font> de tamanho 7 são:

- [o, $\epsilon$, s, $\epsilon$, s, $\epsilon$, o] $\rightarrow$ [o, $\epsilon$, s, $\epsilon$, s, $\epsilon$, o] $\rightarrow$ [o, s, s, o]
- [$\epsilon$, o, s, $\epsilon$, s, $\epsilon$, o] $\rightarrow$ [$\epsilon$, o, s, $\epsilon$, s, $\epsilon$, o] $\rightarrow$ [o, s, s, o]
- [$\epsilon$, o, s, $\epsilon$, s, o, $\epsilon$] $\rightarrow$ [$\epsilon$, o, s, $\epsilon$, s, o, $\epsilon$] $\rightarrow$ [o, s, s, o]
- [o, o, s, $\epsilon$, s, o, $\epsilon$] $\rightarrow$ [o, s, $\epsilon$, s, o, $\epsilon$] $\rightarrow$ [o, s, s, o]

O alinhamento de $X$ e $Y$ funciona da forma *many-to-one*, isto é, diferentes alinhamentos levam ao mesmo $Y$.

---

# **Decoding** 

Vamos supor que $Y = [h, e, l, l, o]$, com tokens $\mathcal{Y} = \{h,e,l,o, \epsilon\}$. A cada *time-step* temos como saída uma distribuição de probabilidade de todos os tokens.

<center><img src="figures/ctc_condional_proba.png" width=400/></center>

Ref: https://distill.pub/2017/ctc/

Agora podemos calcular as probabilidade de uma dada palavra (sequência) acontecer:

- Multiplicar as probabilidade de cada instante $t$ para os diferentes possíveis caminhos.  
- Somar as probabilidades dos caminhso que levam a mesma palavra (sequência).


Seja $Y = [a]$, os tokens $\mathcal{Y} = \{a, \epsilon \}$ e $X$ de tamanho $2$. Temos:

$P(Y|X) = \sum_{A \in A_{X,Y}} \prod p_t(v_t|X) = p_{a,t_1}p_{\epsilon,t_2} + p_{\epsilon,t_1}p_{a,t_2} + p_{a,t_1}p_{a,t_2}$

---

# **Calculando a loss** 

Utilizando programação dinâmica podemos fazer uma busca nesse grafo de forma mais eficiente. Um exemplo visual desse processo para $Y = [a, b]$, os tokens $\mathcal{Y} = \{a, b,\epsilon \}$ e $X$ de tamanho $6$.
<center><img src="figures/example_decoding.png" width=400/></center>

Ref: https://distill.pub/2017/ctc/

Então o nosso modelo retorna uma distribuição de probabilidade para cada instante de tempo $t$. Utiliza-se essa distribuição para estimar a probabilidade da sequência. Tudo isso com o objetivo de calcular a *loss*. Seja $\mathcal{D}$ o conjunto de treino os parâmetros do modelo são tunados para minimizar a seguinte equação:

$$\mathcal{L}(X,Y) = \sum_{(X,Y \in \mathcal{D})} -\log p(Y|X)$$

---

# **Inferência** 

Nesse passo temos um modelo treinado que retorna a melhor distribuição de probabilidades possível, então basta resolver:

$$Y^* = \mathop{\arg \max}\limits_{Y} p(Y|X)$$

Agora precisamos calcular p(Y|X) para todas as possiveis sequências e pegar a melhor? Uma possível técnica é pegar o token de maior probabilidade a cada *step*, técnica conhecida como ***best path decoding***. Esse processo é conhecido como decoding da sequência. Existem várias técnicas de decoding como exemplo, ***beam-search decoding***, ***prefix-search decoding*** e ***token passing***.

---

**Referencias**

- https://distill.pub/2017/ctc/
- https://towardsdatascience.com/intuitively-understanding-connectionist-temporal-classification-3797e43a86c